In [9]:
import os
import xslha
from importlib import import_module
import pandas as pd

# Fetching the slha and python outputs filenames
points = os.listdir("scanpoints")
shorts = [os.path.splitext(point)[0] for point in points]
pys = ["pyoutputs."+sh for sh in shorts]
slhas = [os.path.join("scanpoints", slha) for slha in points]

In [10]:
# Fetching particle masses and widths in slhas and reading python outputs
data = {'mA0':[], 'mHc':[], 'mH0':[], 'deltaM':[], 'dHc':[], 'rmax':[], 'ana':[], 'topo':[], 'bestTx':[], 'pions':[]}
for slha, py in zip(slhas, pys):
    try:
        f = xslha.read(slha)
    except UnicodeDecodeError:
        print("could not read", slha)
    if not "MASS" in f.blocks:
        print("no MASS block in", slha)
        continue
    mA0 = f.Value("MASS", [36])
    mHc = f.Value("MASS", [37])
    mH0 = f.Value("MASS", [35])
    deltaM = mHc - mH0
    brHc = f.br[37]
    # Is H+ decaying to pions or not
    pions = False
    for b in brHc:
        if 211 in b:
            pions = True
            break
    data['pions'].append(pions)
    # Converting widths to ctau in meters
    dHc = 1/f.Value("WIDTH", 37)*6.582E-25*299792458
    data['mA0'].append(mA0)
    data['mHc'].append(mHc)
    data['mH0'].append(mH0)
    data['deltaM'].append(deltaM)
    data['dHc'].append(dHc)
    try:
        dic = import_module(py).smodelsOutput
    except ModuleNotFoundError:
        print("could not load", py)
    res = dic['ExptRes']
    rmax = res[0]['r']
    data['rmax'].append(rmax) 
    data['ana'].append(res[0]['AnalysisID'])
    data['topo'].append(res[0]['TxNames'])
    # Looking for the "best" txname
    weights = res[0]['TxNames weights (fb)']
    bestW = 0
    for tx in weights:
        if weights[tx] > bestW:
            bestW =  weights[tx]
            bestTx = tx
    data['bestTx'].append(bestTx)
df = pd.DataFrame(data)
df.to_csv("scalarData.csv")

KeyError: 'dA0'